# Sagemaker and Seldon Core Scikit-learn Example

 * train a model on AWS Sagemaker
 * Deploy locally on Seldon Core
 
 This tutorial assumes you are already familiar with Sagemaker and have an AWS account you can use.

## Train Scikit-learn Iris Model

We will use the Sagemaker example notebook [Iris Training and Prediction with Sagemaker Scikit-learn](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_iris/Scikit-learn%20Estimator%20Example%20With%20Batch%20Transform.ipynb)

  1. Run this notebook inside Sagemaker but change the training step to include an ```output_path``` parameter with value an S3 bucket you have created. For example:
     * <img src="sagemaker-scikit-iris-train.png"/>
  1. Once trained you should find the model saved to the location you specified. You will need to use this S3 location later when deploying locally.